# Secure Software Engineering Exercise 2

In [1]:
import numpy as np
import pandas as pd
import os

# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
# os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = "C:\Program Files\Git\cmd\git.exe"

import git
import re
# from git import RemoteProgress

from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 1. eclipse-ee4j/mojarra
CVE-ID - CVE-2018-14371<br/>
Project Link:https://github.com/eclipse-ee4j/mojarra<br/>
Commit:1b434748d9239f42eae8aa7d37d7a0930c061e24

### a) What was the message and title of the fixing commit? 

In [3]:
import pprint as pprint
remote_link = "https://github.com/eclipse-ee4j/mojarra"
local_link = "mojarra"
fixing_commit = "1b434748d9239f42eae8aa7d37d7a0930c061e24"

repo = Repo(local_link)
show_data = repo.git.show("--shortstat", fixing_commit).splitlines()
commit = repo.commit(fixing_commit)
print("Commit Title:"+'\x1b[0m'+show_data[4])
print("Commit Message:")
for index in range(len(show_data)):
    if index>5 and index<19:
        print(show_data[index])

Commit Title:    fixing CTS failure issue in master branch
Commit Message:
 2 files changed, 5 insertions(+), 1 deletion(-)


### b) How many total files were affected in the fixing commit?

In [4]:
stats = commit.stats

print("Affected Files:"+ str(stats.total["files"]))

Affected Files:2


### c) How many total directories were affected in the fixing commit? 

In [5]:
totalDirAffected = repo.git.show("--dirstat", fixing_commit).splitlines()
affectedDir = []
for info in totalDirAffected:
    if "% " in info:
        affectedDir.append(info[info.find("% ")+2:])
print(len(affectedDir),"Directories affected: ")
for pline in affectedDir:
    print(pline)

2 Directories affected: 
impl/src/main/java/com/sun/faces/application/applicationimpl/
impl/src/main/java/com/sun/faces/application/resource/


### d) How many total lines of code(including comments and blank lines)  were deleted? 

In [19]:
diffData = repo.git.show("--stat", fixing_commit).splitlines()
#for line in diffData:
#    print(line)
print("Deleted lines(including comment and blank lines):"+diffData[8][35:48])


Deleted lines(including comment and blank lines):1 deletion(-)


### e) How many total lines of code(including comments and blank lines)  were added? 

In [24]:
diffData = repo.git.show("--stat", fixing_commit).splitlines()
#for line in diffData:
#    print(line)
print("Inserted lines(including comment and blank lines):"+diffData[8][18:34])

Inserted lines(including comment and blank lines):5 insertions(+),


### f) How many total lines of code(excluding comments and blank lines)  were deleted? 

In [25]:
diffData = repo.git.diff("-w",fixing_commit + "^", fixing_commit).splitlines()
comment = False
count = 0
for line in diffData:
    if(not line[0:3] == '---'):
        if(line.find('/*') != -1):
            comment = True
            continue
        elif(line.find('*/') != -1):
            comment = False
            continue
        if(comment==False):
            if(line[0:1] == '-' and not re.search(r'\s', line) is None):
                if(line.find('//') == -1):
                    count=count+1
print("Deleted lines(excluding comments and blank lines):"+ str(count))

Deleted lines(excluding comments and blank lines):1


### g) How many total lines of code(excluding comments and blank lines)  were added?

In [26]:
diffData = repo.git.diff("-w",fixing_commit + "^", fixing_commit).splitlines()
comment = False
count = 0
for line in diffData:
    if(not line[0:3] == '+++'):
        if(line.find('/*') != -1):
            comment = True
            continue
        elif(line.find('*/') != -1):
            comment = False
            continue
        if(comment==False):
            if(line[0:1] == '+' and not re.search(r'\s', line) is None):
                if(line.find('//') == -1):
                    count=count+1
print("Inserted lines(excluding comments and blank lines):"+ str(count))

Inserted lines(excluding comments and blank lines):5


### h)  How many days were between the current fixing commit and the previous commit of each affected file? 

In [36]:
from datetime import datetime
from datetime import date

for file in stats.files:   
    print("File: "+file)
    log_data = repo.git.log(fixing_commit,file).splitlines()
    commitID = 0
    date1 = datetime.now()
    date2 = datetime.now()
    for line in log_data:
        #print(line)
        if(line[0:8] == "Date:   "):
            commitID=commitID+1
            if(commitID ==1):
                #date1 = datetime.strptime(line[8:31], '%a %b %d %H:%M:%S %Y')
                print("Current Commit Time: "+str(date1))
                continue
            if(commitID ==2):
                #date2 = datetime.strptime(line[8:31], '%a %b %d %H:%M:%S %Y')
                print("Previous Commit Time: "+str(date2))
                break
    if(commitID == 1):
        print("This file only has one commit.")
        print("----------------------")
    else:
        print("Time Difference: "+str(date1-date2))
        print("----------------------")

File: impl/src/main/java/com/sun/faces/application/applicationimpl/InstanceFactory.java
Current Commit Time: 2019-10-23 20:57:57.149972
Previous Commit Time: 2019-10-23 20:57:57.150091
Time Difference: -1 day, 23:59:59.999881
----------------------
File: impl/src/main/java/com/sun/faces/application/resource/ResourceManager.java
Current Commit Time: 2019-10-23 20:57:57.166638
Previous Commit Time: 2019-10-23 20:57:57.166709
Time Difference: -1 day, 23:59:59.999929
----------------------


### i)  How many time has each affected file of the current fixing commit been modified in the past since their creation?  

In [37]:
for file in stats.files:   
    print("File: "+file)
    log_data = repo.git.log(fixing_commit,"--pretty=\"format:%H%M%S\"", "--",file).splitlines()
    print("Times of modification(including renaming): "+str(len(log_data)))
    print("-------------------------------")

File: impl/src/main/java/com/sun/faces/application/applicationimpl/InstanceFactory.java
Times of modification(including renaming): 2
-------------------------------
File: impl/src/main/java/com/sun/faces/application/resource/ResourceManager.java
Times of modification(including renaming): 2
-------------------------------


### j)  Which developers have modified each affected file since its creation? 

In [38]:
for file in stats.files:   
    print("File: "+file) 
    log_data = repo.git.log(fixing_commit,"--pretty=format:%an", "--follow", "--",file).splitlines()
    x = np.array(log_data) 
    uniqueNames = np.unique(x)
    print("Contributors: ")
    for name in uniqueNames:
        print(name) 
    print()

File: impl/src/main/java/com/sun/faces/application/applicationimpl/InstanceFactory.java
Contributors: 
Zhijun Ren

File: impl/src/main/java/com/sun/faces/application/resource/ResourceManager.java
Contributors: 
Zhijun Ren



### k) For each developer identified, how many commits have each of them submitted? From your observation, are the involving developers experienced or new ones or both?

In [39]:
all_developers = repo.git.shortlog("-sne", "--all").splitlines()

author_commits = []
identifiedDev = []
for file in stats.files:   
    #print("File: "+file) 
    log_data = repo.git.log(fixing_commit,"--pretty=format:%an", "--follow", "--",file).splitlines()
    x = np.array(log_data) 
    uniqueNames = np.unique(x)
    #print("Contributors: ")
    for name in uniqueNames:
        identifiedDev.append(name) 
dev = list(set(identifiedDev))
print("commitTimes"+"\t\t"+"author")
for developer in all_developers:
    #print(developer)
    commitTimes, author = developer.split("\t")
    for tem in dev:
        if tem in author:
            print(commitTimes+"\t\t"+author)


commitTimes		author
    11		Zhijun Ren <ren.zhijun@oracle.com>


## 2. apache/struts
CVE-ID - CVE-2014-0113<br/>
Project Link:  https://github.com/apache/struts<br/>
Commit:  74e26830d2849a84729b33497f729e0f033dc147

### a) What was the message and title of the fixing commit? 

In [76]:
import pprint as pprint
remote_link = "https://github.com/apache/struts"
local_link = "struts"
fixing_commit = "74e26830d2849a84729b33497f729e0f033dc147"

repo = Repo(local_link)
show_data = repo.git.show("--shortstat", fixing_commit).splitlines()
commit = repo.commit(fixing_commit)
print("Commit Title:"+show_data[4])
print("Commit Message:"+show_data[6])

Commit Title:    Adds additional pattern to prevent access to getClass method
Commit Message: 2 files changed, 7 insertions(+), 3 deletions(-)


### b) How many total files were affected in the fixing commit?

In [41]:
commit = repo.commit(fixing_commit)
stats = commit.stats

print("Affected Files:"+ str(stats.total["files"]))

Affected Files:2


### c) How many total directories were affected in the fixing commit? 

In [42]:
totalDirAffected = repo.git.show("--dirstat", fixing_commit).splitlines()
affectedDir = []
for info in totalDirAffected:
    if "% " in info:
        affectedDir.append(info[info.find("% ")+2:])
print(len(affectedDir),"Directories affected: ")
for pline in affectedDir:
    print(pline)

2 Directories affected: 
core/src/main/java/com/opensymphony/xwork2/security/
core/src/test/java/com/opensymphony/xwork2/interceptor/


### d) How many total lines of code(including comments and blank lines)  were deleted? 

In [52]:
diffData = repo.git.show("--stat", fixing_commit).splitlines()
for line in diffData:
    print(line)
print("Deleted lines(including comment and blank lines):"+diffData[8][34:58])


commit 74e26830d2849a84729b33497f729e0f033dc147
Author: Lukasz Lenart <lukaszlenart@apache.org>
Date:   Sun Jan 10 11:46:46 2016 +0100

    Adds additional pattern to prevent access to getClass method

 .../xwork2/security/DefaultExcludedPatternsChecker.java          | 1 +
 .../xwork2/interceptor/ParametersInterceptorTest.java            | 9 ++++++---
 2 files changed, 7 insertions(+), 3 deletions(-)
Deleted lines(including comment and blank lines): 3 deletions(-)


### e) How many total lines of code(including comments and blank lines)  were added? 

In [57]:
diffData = repo.git.show("--stat", fixing_commit).splitlines()
for line in diffData:
    print(line)
print("Inserted lines(including comment and blank lines):"+diffData[8][17:30])

commit 74e26830d2849a84729b33497f729e0f033dc147
Author: Lukasz Lenart <lukaszlenart@apache.org>
Date:   Sun Jan 10 11:46:46 2016 +0100

    Adds additional pattern to prevent access to getClass method

 .../xwork2/security/DefaultExcludedPatternsChecker.java          | 1 +
 .../xwork2/interceptor/ParametersInterceptorTest.java            | 9 ++++++---
 2 files changed, 7 insertions(+), 3 deletions(-)
Inserted lines(including comment and blank lines): 7 insertions


### f) How many total lines of code(excluding comments and blank lines)  were deleted? 

In [58]:
diffData = repo.git.diff("-w",fixing_commit + "^", fixing_commit).splitlines()
comment = False
count = 0
for line in diffData:
    if(not line[0:3] == '---'):
        if(line.find('/*') != -1):
            comment = True
            continue
        elif(line.find('*/') != -1):
            comment = False
            continue
        if(comment==False):
            if(line[0:1] == '-' and not re.search(r'\s', line) is None):
                if(line.find('//') == -1):
                    count=count+1
print("Deleted lines(excluding comments and blank lines):"+ str(count))

Deleted lines(excluding comments and blank lines):3


### g) How many total lines of code(excluding comments and blank lines)  were added?

In [59]:
diffData = repo.git.diff("-w",fixing_commit + "^", fixing_commit).splitlines()
comment = False
count = 0
for line in diffData:
    if(not line[0:3] == '+++'):
        if(line.find('/*') != -1):
            comment = True
            continue
        elif(line.find('*/') != -1):
            comment = False
            continue
        if(comment==False):
            if(line[0:1] == '+' and not re.search(r'\s', line) is None):
                if(line.find('//') == -1):
                    count=count+1
print("Inserted lines(excluding comments and blank lines):"+ str(count))

Inserted lines(excluding comments and blank lines):7


### h)  How many days were between the current fixing commit and the previous commit of each affected file? 

In [60]:
from datetime import datetime
from datetime import date
for file in stats.files:   
    print("File: "+file)
    log_data = repo.git.log(fixing_commit,file).splitlines()
    commitID = 0
    date1 = datetime.now()
    date2 = datetime.now()
    for line in log_data:
        if(line[0:8] == "Date:   "):
            commitID=commitID+1
            if(commitID ==1):
                date1 = datetime.strptime(line[8:38], '%a %b %d %H:%M:%S %Y %z')
                print("Current Commit Time: "+str(date1))
                continue
            if(commitID ==2):
                date2 = datetime.strptime(line[8:38], '%a %b %d %H:%M:%S %Y %z')
                print("Previous Commit Time: "+str(date2))
                break
    if(commitID == 1):
        print("This file only has one commit.")
        print("----------------------")
    else:
        print("Time Difference: "+str(date1-date2))
        print("----------------------")

File: core/src/main/java/com/opensymphony/xwork2/security/DefaultExcludedPatternsChecker.java
Current Commit Time: 2016-01-10 11:46:46+01:00
Previous Commit Time: 2015-09-23 08:42:01+02:00
Time Difference: 109 days, 4:04:45
----------------------
File: core/src/test/java/com/opensymphony/xwork2/interceptor/ParametersInterceptorTest.java
Current Commit Time: 2016-01-10 11:46:46+01:00
Previous Commit Time: 2015-09-23 08:42:01+02:00
Time Difference: 109 days, 4:04:45
----------------------


### i)  How many time has each affected file of the current fixing commit been modified in the past since their creation?  

In [61]:
for file in stats.files:   
    print("File: "+file)
    log_data = repo.git.log(fixing_commit,"--pretty=\"format:%H%M%S\"", "--",file).splitlines()
    print("Times of modification(including renaming): "+str(len(log_data)))
    print("-------------------------------")

File: core/src/main/java/com/opensymphony/xwork2/security/DefaultExcludedPatternsChecker.java
Times of modification(including renaming): 3
-------------------------------
File: core/src/test/java/com/opensymphony/xwork2/interceptor/ParametersInterceptorTest.java
Times of modification(including renaming): 3
-------------------------------


### j)  Which developers have modified each affected file since its creation? 

In [62]:
for file in stats.files:   
    print("File: "+file) 
    log_data = repo.git.log(fixing_commit,"--pretty=format:%an", "--follow", "--",file).splitlines()
    x = np.array(log_data) 
    uniqueNames = np.unique(x)
    print("Contributors: ")
    for name in uniqueNames:
        print(name) 
    print()

File: core/src/main/java/com/opensymphony/xwork2/security/DefaultExcludedPatternsChecker.java
Contributors: 
Johannes Geppert
Lukasz Lenart

File: core/src/test/java/com/opensymphony/xwork2/interceptor/ParametersInterceptorTest.java
Contributors: 
Johannes Geppert
John Lindal
Lukasz Lenart
Maurizio Cucchiara
René Gielen



### k) For each developer identified, how many commits have each of them submitted? From your observation, are the involving developers experienced or new ones or both?

In [63]:
all_developers = repo.git.shortlog("-sne", "--all").splitlines()

author_commits = []
identifiedDev = []
for file in stats.files:   
    #print("File: "+file) 
    log_data = repo.git.log(fixing_commit,"--pretty=format:%an", "--follow", "--",file).splitlines()
    x = np.array(log_data) 
    uniqueNames = np.unique(x)
    #print("Contributors: ")
    for name in uniqueNames:
        identifiedDev.append(name) 
dev = list(set(identifiedDev))
print("commitTimes"+"\t\t"+"author")
for developer in all_developers:
    #print(developer)
    commitTimes, author = developer.split("\t")
    for tem in dev:
        if tem in author:
            print(commitTimes+"\t\t"+author)


commitTimes		author
  2224		Lukasz Lenart <lukaszlenart@apache.org>
   241		Lukasz Lenart <lukasz.lenart@gmail.com>
   189		René Gielen <rgielen@apache.org>
   115		Maurizio Cucchiara <mcucchiara@apache.org>
   103		Johannes Geppert <jogep@apache.org>
    49		Johannes Geppert <johgep@gmail.com>
    31		John Lindal <jafl@apache.org>
     3		Johannes Geppert <post@jgeppert.com>


## 2. jenkinsci/groovy-sandbox
CVE-ID - CVE-2018-1000865<br/>
Project Link:https://github.com/jenkinsci/groovy-sandbox<br/>
Commit:0cd7ec12b7c56cfa3167d99c5f43147ce05449d3

### a) What was the message and title of the fixing commit? 

In [77]:
import pprint as pprint
remote_link = "https://github.com/jenkinsci/groovy-sandbox"
local_link = "groovy-sandbox"
fixing_commit = "0cd7ec12b7c56cfa3167d99c5f43147ce05449d3"

repo = Repo(local_link)
show_data = repo.git.show("--shortstat", fixing_commit).splitlines()
print("Commit Title:"+'\x1b[0m'+show_data[4])
print("Commit Message:"+'\x1b[0m'+show_data[6])

Commit Title:    [SECURITY-1186] Forbid sandboxed classes from overriding finalize.
Commit Message: 2 files changed, 180 insertions(+)


### b) How many total files were affected in the fixing commit?

In [65]:
commit = repo.commit(fixing_commit)
stats = commit.stats

print("Affected Files:"+ str(stats.total["files"]))

Affected Files:2


### c) How many total directories were affected in the fixing commit? 

In [66]:
totalDirAffected = repo.git.show("--dirstat", fixing_commit).splitlines()
affectedDir = []
for info in totalDirAffected:
    if "% " in info:
        affectedDir.append(info[info.find("% ")+2:])
print(len(affectedDir),"Directories affected: ")
for pline in affectedDir:
    print(pline)

2 Directories affected: 
src/main/java/org/kohsuke/groovy/sandbox/
src/test/java/org/kohsuke/groovy/sandbox/


### d) How many total lines of code(including comments and blank lines)  were deleted? 

In [67]:
diffData = repo.git.show("--stat", fixing_commit).splitlines()
#for line in diffData:
#    print(line)
print("Deleted lines(including comment and blank lines):"+diffData[8][35:37])


Deleted lines(including comment and blank lines):


### e) How many total lines of code(including comments and blank lines)  were added? 

In [68]:
diffData = repo.git.show("--stat", fixing_commit).splitlines()
#for line in diffData:
#    print(line)
print("Inserted lines(including comment and blank lines):"+diffData[8][17:20])


Inserted lines(including comment and blank lines): 18


### f) How many total lines of code(excluding comments and blank lines)  were deleted? 

In [69]:
diffData = repo.git.diff("-w",fixing_commit + "^", fixing_commit).splitlines()
comment = False
count = 0
for line in diffData:
    if(not line[0:3] == '---'):
        if(line.find('/*') != -1):
            comment = True
            continue
        elif(line.find('*/') != -1):
            comment = False
            continue
        if(comment==False):
            if(line[0:1] == '-' and not re.search(r'\s', line) is None):
                if(line.find('//') == -1):
                    count=count+1
print("Deleted lines(excluding comments and blank lines):"+ str(count))

Deleted lines(excluding comments and blank lines):0


### g) How many total lines of code(excluding comments and blank lines)  were added?

In [70]:
diffData = repo.git.diff("-w",fixing_commit + "^", fixing_commit).splitlines()
comment = False
count = 0
for line in diffData:
    if(not line[0:3] == '+++'):
        if(line.find('/*') != -1):
            comment = True
            continue
        elif(line.find('*/') != -1):
            comment = False
            continue
        if(comment==False):
            if(line[0:1] == '+' and not re.search(r'\s', line) is None):
                if(line.find('//') == -1):
                    count=count+1
print("Inserted lines(excluding comments and blank lines):"+ str(count))

Inserted lines(excluding comments and blank lines):121


### h)  How many days were between the current fixing commit and the previous commit of each affected file? 

In [71]:
from datetime import datetime
from datetime import date
for file in stats.files:   
    print("File: "+file)
    log_data = repo.git.log(fixing_commit,file).splitlines()
    commitID = 0
    date1 = datetime.now()
    date2 = datetime.now()
    for line in log_data:
        if(line[0:8] == "Date:   "):
            commitID=commitID+1
            if(commitID ==1):
                date1 = datetime.strptime(line[8:38], '%a %b %d %H:%M:%S %Y %z')
                print("Current Commit Time: "+str(date1))
                continue
            if(commitID ==2):
                date2 = datetime.strptime(line[8:38], '%a %b %d %H:%M:%S %Y %z')
                print("Previous Commit Time: "+str(date2))
                break
    if(commitID == 1):
        print("This file only has one commit.")
        print("----------------------")
    else:
        print("Time Difference: "+str(date1-date2))
        print("----------------------")

File: src/main/java/org/kohsuke/groovy/sandbox/SandboxTransformer.java
Current Commit Time: 2018-10-17 20:14:28-04:00
Previous Commit Time: 2017-08-16 14:15:44-04:00
Time Difference: 427 days, 5:58:44
----------------------
File: src/test/java/org/kohsuke/groovy/sandbox/FinalizerTest.java
Current Commit Time: 2018-10-17 20:14:28-04:00
This file only has one commit.
----------------------


### i)  How many time has each affected file of the current fixing commit been modified in the past since their creation?  

In [72]:
for file in stats.files:   
    print("File: "+file)
    log_data = repo.git.log(fixing_commit,"--pretty=\"format:%H%M%S\"", "--",file).splitlines()
    print("Times of modification(including renaming): "+str(len(log_data)))
    print("-------------------------------")

File: src/main/java/org/kohsuke/groovy/sandbox/SandboxTransformer.java
Times of modification(including renaming): 12
-------------------------------
File: src/test/java/org/kohsuke/groovy/sandbox/FinalizerTest.java
Times of modification(including renaming): 1
-------------------------------


### j)  Which developers have modified each affected file since its creation? 

In [73]:
for file in stats.files:   
    print("File: "+file) 
    log_data = repo.git.log(fixing_commit,"--pretty=format:%an", "--follow", "--",file).splitlines()
    x = np.array(log_data) 
    uniqueNames = np.unique(x)
    print("Contributors: ")
    for name in uniqueNames:
        print(name) 
    print()

File: src/main/java/org/kohsuke/groovy/sandbox/SandboxTransformer.java
Contributors: 
Andrew Bayer
Devin Nusbaum
Jesse Glick
Julien Viet
Kohsuke Kawaguchi
Oliver Gondža

File: src/test/java/org/kohsuke/groovy/sandbox/FinalizerTest.java
Contributors: 
Devin Nusbaum



### k) For each developer identified, how many commits have each of them submitted? From your observation, are the involving developers experienced or new ones or both?

In [74]:
all_developers = repo.git.shortlog("-sne", "--all").splitlines()

author_commits = []
identifiedDev = []
for file in stats.files:   
    #print("File: "+file) 
    log_data = repo.git.log(fixing_commit,"--pretty=format:%an", "--follow", "--",file).splitlines()
    x = np.array(log_data) 
    uniqueNames = np.unique(x)
    #print("Contributors: ")
    for name in uniqueNames:
        identifiedDev.append(name) 
dev = list(set(identifiedDev))
print("commitTimes"+"\t\t"+"author")
for developer in all_developers:
    #print(developer)
    commitTimes, author = developer.split("\t")
    for tem in dev:
        if tem in author:
            print(commitTimes+"\t\t"+author)


commitTimes		author
   116		Kohsuke Kawaguchi <kk@kohsuke.org>
    40		Jesse Glick <jglick@cloudbees.com>
    29		Andrew Bayer <andrew.bayer@gmail.com>
    11		Devin Nusbaum <d.w.nusbaum@gmail.com>
     2		Julien Viet <julien@julienviet.com>
     1		Devin Nusbaum <dwnusbaum@users.noreply.github.com>
     1		Oliver Gondža <ogondza@gmail.com>
